<a href="https://colab.research.google.com/github/Saha555339/Coursework-3rd-year/blob/main/%D0%9A%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F_%D0%9A%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIC (minepy)

In [1]:
!pip install minepy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.0/497.0 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for minepy: filename=minepy-1.2.6-cp39-cp39-linux_x86_64.whl size=226934 sha256=db600d88aa51f722fe930a3ff41e98035fb757c3c1b0a7c5f83bc1b718a30691
  Stored in directory: /root/.cache/pip/wheels/36/71/63/ac6efc0d0f43ad93398a7b25b6ef3b3aacf109e7dc5afe588f
Successfully built minepy


In [2]:
import minepy
import pandas as pd

data = pd.read_csv("/content/Data_Deputies_Rating_2021_June_Passed.csv")
data.head()

,Unnamed: 0,ФИО,Активность в ГД,Индекс народного голосования,Оценка экспертов,Индекс цитируемости,КПДГД,Прошел
0,0,Крашенинников Павел Владимирович,9.32,8.66,29.5,10.94,58.42,1
1,1,Хинштейн Александр Евсеевич,5.22,0.48,24.0,28.45,58.15,1
2,2,Нилов Ярослав Евгеньевич,14.91,10.63,24.0,8.21,57.75,1
3,3,Толстой Петр Олегович,0.37,13.27,29.0,13.87,56.51,1
4,4,Исаев Андрей Константинович,4.85,11.60,27.0,12.92,56.37,1


In [3]:
def print_stats(mine):
    print("MIC", mine.mic())
    print("MAS", mine.mas())
    print("MEV", mine.mev())
    print("GMIC", mine.gmic())
    print("TIC", mine.tic())

***MIC (Maximal Information Coefficient)*** - это мера взаимной информации, которая измеряет степень нелинейной зависимости между двумя переменными. MIC находится в диапазоне от 0 до 1, где 0 означает отсутствие зависимости, а 1 - идеальную зависимость.


---


***MAS (Maximal Asymmetric Surprisal)*** - это мера взаимной информации, которая измеряет количество информации, которую переменная X сообщает о переменной Y, но которую Y не сообщает о X. MAS находится в диапазоне от 0 до бесконечности, где 0 означает отсутствие зависимости.


---


***MEV (Maximal Eigenvalue of the covariance matrix)*** - это мера, которая используется для оценки степени линейной зависимости между переменными. MEV также находится в диапазоне от 0 до 1, где 0 означает отсутствие зависимости, а 1 - идеальную зависимость.


---



***GMIC (Generalized Maximal Information Coefficient)*** - это усовершенствованная версия MIC, которая учитывает не только нелинейную зависимость между переменными, но и корреляцию между ними. GMIC также находится в диапазоне от 0 до 1, где 0 означает отсутствие зависимости, а 1 - идеальную зависимость.


---


***TIC (Total Information Coefficient)*** - это мера, которая измеряет общую степень зависимости между двумя переменными, включая линейную и нелинейную зависимость, а также зависимость от других переменных. TIC находится в диапазоне от 0 до 1, где 0 означает отсутствие зависимости, а 1 - идеальном случае.

In [5]:
import numpy as np

mine = minepy.MINE(alpha=0.5, c=3, est="mic_approx")

X = np.asarray(data['Индекс народного голосования'])
Y = np.asarray(data['Прошел'])

mine.compute_score(X,Y)

print_stats(mine)

MIC 0.041890553280087804
MAS 0.03843063088069103
MEV 0.041890553280087804
GMIC 0.02488813120691084
TIC 0.4186607062715392


Параметр ***'alpha'*** контролирует чувствительность MINE к нелинейным зависимостям между переменными. Значение alpha по умолчанию равно 0.6, но для данных с рейтингом депутатов и значениями {1, 0} можно принять значение равное 0.5

---


Параметр ***'c'*** отвечает за количество бинов, используемых при расчете MINE. Оптимальное значение для 438 записей в таблице было принято за 5.


---


Параметр ***'est'*** отвечает за метод оценки MINE и может принимать значения *'mic_approx'*, *'mic_e'*, *'mas'*, *'gmic'*, *'tic'*, 'mcpp'. Для данных с рейтингом депутатов и значениями {1, 0} можно начать с метода **'mic_approx'**, который является быстрой и точной оценкой MINE для данных с малым числом уникальных значений.

In [6]:
from scipy.stats import norm


corr = mine.mic()
df = len(X) - 2
t = corr * ((len(X) - 2) / ((1 - corr**2) * df))**0.5
p_value = 2 * (1 - norm.cdf(abs(t)))

print("p-value:", p_value)

p-value: 0.9665566079294514


Здесь мы используем стандартный подход, когда ***p-value*** вычисляется как вероятность получения значения статистики, не меньше, чем наблюдаемое значение, при условии, что корреляции между двумя переменными нет (нулевая гипотеза). Для этого мы вычисляем статистику t-распределения, используя коэффициент корреляции, количество наблюдений и количество степеней свободы. Затем мы используем функцию norm.cdf() из библиотеки SciPy для вычисления вероятности получения значения статистики меньше, чем наблюдаемое значение.


---

Значение ***p_value*** очень близко к 1, значит мы можем с уверенностью сказать, что нулевая гипотеза принимается с достаточно большой вероятностью и никакой корреляции на данной выборке данных выявлено не было.

# Коеффициент Фишера

In [7]:
from scipy.stats import pointbiserialr

corr, p_value = pointbiserialr(X, Y)

print("Коэффициент корреляции Фишера: ", corr)
print("p_value: ", p_value)

Коэффициент корреляции Фишера:  0.050578526182084985
p_value:  0.2908853697729455


***p_value*** - это вероятность получить такое или еще более экстремальное значение коэффициента корреляции, если бы нулевая гипотеза о том, что коэффициент равен 0, была верна. То есть, если p-значение мало, это говорит о том, что есть значимая корреляция между двумя переменными.


---


**Коэффициент корреляции Фишера** (corr) принимает значения от -1 до 1, где:
*   1 - полная положительная корреляция
*   0 - отсутствие корреляции
*  -1 - полная отрицательная корреляций

Значение коэффициента корреляции Фишера интерпретируется как мера силы связи между двумя переменными, где значения близкие к 1 или -1 указывают на сильную связь, а значения близкие к 0 указывают на отсутствие связи.


---

***p_value*** равно 0.29, что означает вероятность случайной ошибки при отвержении нулевой гипотезы достигает почти 30%, что достаточно велико для того, чтобы мы могли сказать, что никакой связи между значениями на данной выборке не наблюдается.

# Оценка зависимости с помощью логистической регрессии.

In [8]:
import statsmodels.api as sm

y = Y.reshape(-1, 1)
XLog = sm.add_constant(X.reshape(-1, 1))

model = sm.Logit(y, XLog).fit()

print(model.summary())

Optimization terminated successfully.
         Current function value: 0.689199
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  438
Model:                          Logit   Df Residuals:                      436
Method:                           MLE   Df Model:                            1
Date:                Thu, 13 Apr 2023   Pseudo R-squ.:                0.001850
Time:                        10:23:12   Log-Likelihood:                -301.87
converged:                       True   LL-Null:                       -302.43
Covariance Type:            nonrobust   LLR p-value:                    0.2901
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2888      0.166     -1.742      0.082      -0.614       0.036
x1             0.0381      0.